In [10]:
from math import *
import numpy as np
from scipy.stats import levy_stable
from scipy.stats import uniform
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt

In [184]:
def prior_sample(seed):
    # Sample from prior distribution
    # For univariate alpha-stable models, sample each parameter from its respective uniform distribution
    np.random.seed(seed)
    alpha = uniform.rvs(1.1, 2)  # U[1.1, 2]
    beta = uniform.rvs(-1, 1)  # U[-1, 1]
    gamma = uniform.rvs(0, 300)  # U[0, 300]
    delta = uniform.rvs(-300, 300)  # U[-300, 300]
    return [alpha, beta, gamma, delta]

In [185]:
def univ_alpha_stable_sampler(params,size,seed) :
    alpha,beta,gamma,delta = params[0],params[1],params[2],params[3]
    #print(alpha,beta,gamma,delta)
    y_bar = 0
    np.random.seed(seed)
    w = np.random.standard_exponential(size=size)
    u = np.random.uniform(low = -np.pi/2, high = np.pi/2, size = size)
    if alpha == 1 :
        y_bar = 2/np.pi*((np.pi/2+beta*u)*np.tan(u)-beta*np.log((np.pi/2*w*np.cos(u))/(np.pi/2+beta*u)))
        return gamma*y_bar + delta
    else :
        S = (1+beta**2*np.tan(np.pi*alpha/2)**2)**(1/(2*alpha))
        B = 1/alpha*np.arctan(beta*np.tan(np.pi*alpha/2))
        #print(S,B)
        y_bar = (S*(np.sin(alpha)*(u+B))*(np.cos(u-alpha*(u+B))/w)**((1-alpha)/alpha))/np.cos(u)**(1/alpha)
        return gamma*y_bar + delta

In [256]:
def prior(theta):
    if 1.1 <= theta[0] <= 2.0 and -1 <= theta[1] <=2 and 0 <= theta[2] <= 300 and -300 <= theta[3] <= 300:
        return 1
    else:
        return 0

In [261]:
def transition(theta, cov):
    #theta_new=[]
    #for i in range(len(theta)):
       #theta_new.append(np.random.normal(loc=theta[i], scale=cov[i][i]))
    #return np.array(theta_new)
    return  multivariate_normal.rvs(mean=theta, cov=cov)

In [188]:
def compute_quantiles(data):
    # Compute quantile-based summary statistics
    quantiles = np.percentile(data, [5, 25, 50, 75, 95])
    v_alpha = (quantiles[4] - quantiles[0]) / (quantiles[3] - quantiles[1])
    v_beta = (quantiles[4] + quantiles[0] - 2 * quantiles[2]) / (quantiles[4] - quantiles[0])
    v_gamma = (quantiles[3] - quantiles[1]) / quantiles[2]
    return v_alpha, v_beta, v_gamma

In [189]:
def S(data):
    # Compute summary statistics of the data
    # This function should return low-dimensional summary statistics S(data)
    # For univariate alpha-stable models, use quantile-based estimators along with the mean of the data
    v_alpha, v_beta, v_gamma = compute_quantiles(data)
    mean_x = np.mean(data)
    return np.array([v_alpha, v_beta, v_gamma, mean_x])

In [190]:
def sigma_hat(theta_hat,n_draws = 1000):
    x = np.array([univ_alpha_stable_sampler(theta_hat, size = 200, seed=None) for _ in range(n_draws)])
    sumary_statistics = np.array([S(x_i) for x_i in x])

    return np.cov(sumary_statistics, rowvar=False)

In [191]:
def norm2(u) :
    return np.sqrt(np.sum(u*u))

In [192]:
def K(u, sigma) :
    d = sigma.shape[0]
    #print(d)
    return np.exp(-(u.T@np.linalg.inv(sigma)@u)/2)/(np.sqrt(2*np.pi)**d*np.sqrt(np.linalg.det(sigma)))

In [259]:
alpha = 1.7
beta = 0.9
gamma = 10
delta = 10

def MCMC(N, size, covariance=np.diag([0.25, 0.25, 1, 1]), seed=42):
    epsilons = [k for k in range(1000, 100, -100)]+[k for k in range(100,9, -1)] + [k+0.5 for k in range(9,4,-1)] + [5-k*0.05 for k in range(40)]+[3-0.01*k for k in range(301)]
    y = levy_stable.rvs(alpha, beta, loc=gamma, scale=delta, size=size)
    theta_0 = prior_sample(seed)
    thetas = [theta_0] * N
    n = 0
    n_accepted = 0
    while n<N-1:
        theta = np.array(thetas[n])
        theta_new = transition(theta, cov=covariance)
        #print(theta_new)
        x = univ_alpha_stable_sampler(theta, size=size, seed=seed)
        x_new = univ_alpha_stable_sampler(theta_new, size=size, seed=seed)
        prior1 = prior(theta)
        prior2 = prior(theta_new)
        sigma1 = sigma_hat(theta, n_draws = 1000)
        sigma2 = sigma_hat(theta_new, n_draws = 1000)
        epsilon = epsilons[n]
        #epsilon = 1000
        h = min(1, (prior2*K((S(x_new)-S(y)/epsilon), sigma2))/(prior1*K((S(x)-S(y)/epsilon), sigma1)))
        if prior2 == 0:
            h = 0
        print((prior2,prior1), (K((S(x_new)-S(y)/epsilon), sigma2), K((S(x)-S(y)/epsilon), sigma1)), h)
        #h = min(1, prior2/prior1)
        u = np.random.uniform(0.0, 1.0)
        if u < h :
            thetas[n+1] = theta_new
        else:
            thetas[n+1] = theta
        n += 1
    return thetas

In [262]:
print(MCMC(200, 100))

/tmp/ipykernel_329/1118787516.py:15: RuntimeWarning: invalid value encountered in power
  y_bar = (S*(np.sin(alpha)*(u+B))*(np.cos(u-alpha*(u+B))/w)**((1-alpha)/alpha))/np.cos(u)**(1/alpha)


(0, 1) (nan, 1.914854609162138e-18) 0
(1, 1) (1.7643144785574872e-13, 5.6703784881464e-17) 1
(1, 1) (1.915516682563688e-13, 4.445270543758049e-13) 0.4309111591089578
(1, 1) (4.8747421006631215e-17, 8.08532416923894e-15) 0.00602912387756739
(1, 1) (6.870097909130347e-11, 9.078758230922984e-14) 1
(0, 1) (9.260260778408764e-30, 2.130333301194395e-11) 0
(1, 1) (2.6861490456453083e-16, 4.32868096119865e-11) 6.205467831247812e-06
(1, 1) (4.256628551396722e-15, 9.344229898718995e-12) 0.0004555355120254763
(1, 1) (4.87749895879802e-18, 2.6192956610318952e-11) 1.8621414265530015e-07
(1, 1) (7.495507845077052e-08, 1.8192925887445757e-11) 1
(0, 1) (nan, 6.927302407921151e-09) 0
(0, 1) (nan, 1.5836624307151275e-07) 0
(0, 1) (nan, 7.028199963381414e-08) 0
(1, 1) (1.444016830779712e-14, 1.4942886208064197e-08) 9.663573761275265e-07
(0, 1) (2.9493853284171786e-19, 1.0957441900348368e-07) 0
(0, 1) (nan, 2.2207915472982035e-08) 0
(1, 1) (4.645861682632746e-14, 6.177115985710345e-08) 7.521085395482482e-